In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Activation, Conv2DTranspose
from tensorflow.keras.models import Model

def build_unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Contracting Path (Encoder)
    # Block 1
    c1 = Conv2D(64, (3, 3), padding='same')(inputs)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(64, (3, 3), padding='same')(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    # Block 2
    c2 = Conv2D(128, (3, 3), padding='same')(p1)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(128, (3, 3), padding='same')(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Block 3 - example for additional depth
    c3 = Conv2D(256, (3, 3), padding='same')(p2)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(256, (3, 3), padding='same')(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    # Expansive Path (Decoder)
    # Block 1
    u1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u1 = Concatenate()([u1, c2])
    c4 = Conv2D(128, (3, 3), padding='same')(u1)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(128, (3, 3), padding='same')(c4)
    c4 = Activation('relu')(c4)

    # Block 2
    u2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u2 = Concatenate()([u2, c1])
    c5 = Conv2D(64, (3, 3), padding='same')(u2)
    c5 = Activation('relu')(c5)
    c5 = Conv2D(64, (3, 3), padding='same')(c5)
    c5 = Activation('relu')(c5)

    # Output layer
    outputs = Conv2D(3, (1, 1), activation='sigmoid')(c5)  # Modify the number of filters based on the number of classes

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Example usage
model = build_unet((256, 256, 3))

def psnr_metric(y_true, y_pred):
    """
    Peak Signal-to-Noise Ratio metric.
    """
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

def ssim_metric(y_true, y_pred):
    """
    Custom metric for calculating the Structural Similarity Index (SSIM)
    between two images. Ensures both inputs are float32.
    """
    y_true_f32 = tf.cast(y_true, tf.float32)
    y_pred_f32 = tf.cast(y_pred, tf.float32)
    return tf.image.ssim(y_true_f32, y_pred_f32, max_val=1.0)


model.compile(loss='mean_squared_error',
              metrics=[psnr_metric, ssim_metric])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 256, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 activation_10 (Activation)  (None, 256, 256, 64)         0         ['conv2d_11[0][0]']           
                                                                                                  
 conv2d_12 (Conv2D)          (None, 256, 256, 64)         36928     ['activation_10[0][0]']       
                                                                                            

In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import tensorflow as tf
import os

def decode_img(img, image_size=(256, 256)):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_size)
    img /= 255.0  # Normalize to [0,1]
    return img

def process_path(clear_path, hazy_path):
    print(f"Processing: {clear_path} and {hazy_path}")  # Print the paths being processed
    clear_img = tf.io.read_file(clear_path)
    clear_img = decode_img(clear_img)
    hazy_img = tf.io.read_file(hazy_path)
    hazy_img = decode_img(hazy_img)
    return hazy_img, clear_img

def create_dataset(dir_pairs, image_size=(256, 256), batch_size=32, shuffle=True):
    clear_paths, hazy_paths = [], []

    print("Collecting image paths...")
    # Collect paths of both clear and corresponding hazy images
    for clear_dir, hazy_dir in dir_pairs:
        # Assuming file names match except for the "foggy_beta" part
        for file_name in sorted(os.listdir(clear_dir)):
            if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            base_name = '_'.join(file_name.split('_')[:-1])
            for beta in ["0.01", "0.02", "0.005"]:
                hazy_file_name = f"{base_name}_leftImg8bit_foggy_beta_{beta}.png"
                hazy_path = os.path.join(hazy_dir, hazy_file_name)
                if os.path.exists(hazy_path):
                    clear_paths.append(os.path.join(clear_dir, file_name))
                    hazy_paths.append(hazy_path)

    print(f"Collected {len(clear_paths)} pairs of images.")

    # Create a tf.data.Dataset from paths
    dataset = tf.data.Dataset.from_tensor_slices((clear_paths, hazy_paths))
    print("Creating dataset from paths...")
    dataset = dataset.map(lambda x, y: process_path(x, y), num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        print("Shuffling dataset...")
        dataset = dataset.shuffle(buffer_size=len(clear_paths))

    print("Batching dataset...")
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    print("Dataset created and ready for use.")
    return dataset

# Define the pairs of directories
dir_pairs = [
    ('/content/drive/My Drive/diss/myproj/data_train/clear_images/strasbourg', '/content/drive/My Drive/diss/myproj/data_train/hazy_images/strasbourg'),
    ('/content/drive/My Drive/diss/myproj/data_train/clear_images/hamburg', '/content/drive/My Drive/diss/myproj/data_train/hazy_images/hamburg'),
    ('/content/drive/My Drive/diss/myproj/data_train/clear_images/aachen', '/content/drive/My Drive/diss/myproj/data_train/hazy_images/aachen'),
    ('/content/drive/My Drive/diss/myproj/data_train/clear_images/hanover', '/content/drive/My Drive/diss/myproj/data_train/hazy_images/hanover')
]

# Create and use the TensorFlow dataset
print("Starting dataset creation...")
dataset = create_dataset(dir_pairs)

# Display some info about the dataset (optional)
for hazy_images, clear_images in dataset.take(1):
    print(f"Sample batch - Hazy images shape: {hazy_images.shape}, dtype: {hazy_images.dtype}")
    print(f"Sample batch - Clear images shape: {clear_images.shape}, dtype: {clear_images.dtype}")

#####################

import tensorflow as tf

# Clear any previous session
tf.keras.backend.clear_session()

# Set the mixed precision policy
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('float32')


from tensorflow.keras.optimizers import Adam
#from dehaze import model, build_unet, my_loss, psnr_metric, ssim_metric  # Adjust import according to your file structure
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#from dataset import hazy_images, clear_images_matched  # Assuming these are loaded and prepared as shown
import matplotlib.pyplot as plt
from tensorflow.image import psnr, ssim

# Assuming `dataset` is your complete dataset returned from `create_dataset`
# First, let's count the number of items in the dataset
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

# Split the dataset into training and validation sets
full_dataset = dataset.shuffle(buffer_size=dataset_size)
train_dataset = full_dataset.take(train_size)
val_dataset = full_dataset.skip(train_size)

# Continue with your model definition and training as before
# Ensure you use `train_dataset` and `val_dataset` for training and validation, respectively.



def create_dataset_from_paths(hazy_paths, clear_paths, image_size=(256, 256), batch_size=32, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((hazy_paths, clear_paths))
    dataset = dataset.map(lambda x, y: process_path(x, y), num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(hazy_paths))

    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


# Load and prepare your data
# Ensure datasets are TensorFlow Dataset objects correctly batched
def prepare_tf_dataset(hazy_images, clear_images, batch_size=6):
    dataset = tf.data.Dataset.from_tensor_slices((hazy_images, clear_images))
    dataset = dataset.shuffle(buffer_size=len(hazy_images)).batch(batch_size)
    return dataset

# Split the data into training and validation sets
train_size = int(0.8 * len(hazy_images))
train_hazy, train_clear = hazy_images[:train_size], clear_images[:train_size]
val_hazy, val_clear = hazy_images[train_size:], clear_images[train_size:]

# Prepare TensorFlow datasets
train_dataset = prepare_tf_dataset(train_hazy, train_clear, batch_size=6)
val_dataset = prepare_tf_dataset(val_hazy, val_clear, batch_size=6)


# Assume load_datasets returns properly prepared and normalized TensorFlow Dataset objects
#train_dataset, val_dataset = load_datasets()

# Build the model
model = build_unet((256, 256, 3))

# Define a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)

# Compile the model using Mean Squared Error (MSE) as the loss function
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mean_squared_error',  # Using built-in MSE loss
              metrics=[psnr_metric, ssim_metric])



# Custom callback for epoch printing
class TrainingPrint(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting Epoch {epoch+1}")


early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    mode='min')



# Train the model
history = model.fit(
    train_dataset,
    epochs = 30,
    validation_data=val_dataset,
    callbacks=[ early_stopping_callback, TrainingPrint()])


# Save the final model
model.save('/content/drive/My Drive/diss/myproj/results/3_0BasicUNet.keras')

# Evaluate the model on the validation dataset
val_loss, val_psnr, val_ssim = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}, Validation PSNR: {val_psnr}, Validation SSIM: {val_ssim}")

# Plotting the training history (loss, PSNR, SSIM)
# You can use the plotting code you've provided to visualize the training and validation metrics over epochs.



Mounted at /content/drive
Starting dataset creation...
Collected 2949 pairs of images.
Creating dataset from paths...
Processing: Tensor("args_0:0", shape=(), dtype=string) and Tensor("args_1:0", shape=(), dtype=string)
Shuffling dataset...
Batching dataset...
Dataset created and ready for use.
Sample batch - Hazy images shape: (32, 256, 256, 3), dtype: <dtype: 'float32'>
Sample batch - Clear images shape: (32, 256, 256, 3), dtype: <dtype: 'float32'>
Starting Epoch 1
Epoch 1/30
5/5 [==============================] - 12s 469ms/step - loss: 0.0680 - psnr_metric: 11.8090 - ssim_metric: 0.4086 - val_loss: 0.0583 - val_psnr_metric: 12.4977 - val_ssim_metric: 0.4218
Starting Epoch 2
Epoch 2/30
5/5 [==============================] - 0s 57ms/step - loss: 0.0633 - psnr_metric: 12.1413 - ssim_metric: 0.4082 - val_loss: 0.0537 - val_psnr_metric: 12.9413 - val_ssim_metric: 0.4189
Starting Epoch 3
Epoch 3/30
5/5 [==============================] - 0s 57ms/step - loss: 0.0575 - psnr_metric: 12.6179 -